<a href="https://colab.research.google.com/github/daffamaulana03/Fuzzy-Logic/blob/master/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import random
import numpy
import pandas
import csv

# **BACA DATA**

In [0]:
def readFile():
    data = open("influencers.csv", "r")
    datata = data.readlines()
    
    array = []


    for i in range(1, len(datata)):
      array.append(datata[i].replace('\n','').split(','))
    # print(array[1][1])
    # print(len(array))
    return array

# **FUZZIFIKASI**

Atribut Followers

In [0]:
def fuzzificationFollowers():
  data = readFile()
  followers = []
  # FOLLOWERS

  # Upper	: >60K NOT Upper : <=50K
  # Middle	: 40K<=MID<=50K NOT Mid : <=20K & >80K
  # Bottom	: <=20K NOT Bottom : >45K

  Upper = 60000
  NotUpper = 55000
  Middle1 = 40000
  Middle2 = 50000
  Bottom = 20000
  NotBottom = 45000

  for i in range(len(data)):
    # for j in range(len(data[i])):
    x = int(data[i][1])
    #UPPER
    if (x > Upper):
      Up = 1
    elif (x <= NotUpper):
      Up = 0
    else:
      Up = round((x-NotUpper)/(Upper-NotUpper),2)

    #BOTTOM
    if (x <= Bottom):
      Bot = 1
    elif (x > NotBottom):
      Bot = 0
    else:
      Bot = round((NotBottom-x)/(NotBottom-Bottom),2)

    #MIDDLE
    if ((x > Middle1) & (x <= Middle2)):
      Mid = 1
    elif ((x <= Bottom) or (x > Upper)):
      Mid = 0
    else:
      if (x > Middle2):
        Mid = round((Upper-x)/(Upper-Middle2),2)
      elif (x <= Middle1):
        Mid = round((x-Bottom)/(Middle1-Bottom),2) 
    
    fol = []
    foll = []
    fol.append(Up)
    fol.append('upper')
    foll.append(fol)
    fol = []
    fol.append(Mid)
    fol.append('middle')
    foll.append(fol)
    fol = []
    fol.append(Bot)
    fol.append('bottom')
    foll.append(fol)
    fol = []
    followers.append(foll)
  return followers

Atribut Engagement Rate

In [0]:
def fuzzificationEngageRate():
  data = readFile()
  EngageRate = []
  # ENGAGEMENT RATE

  # High	: >8 NOT High : <=5
  # Average	: 4<AVG<5 NOT Mid : <=2 & >8
  # Low	: <=2 NOT Low : >4.5
  High = 8
  NotHigh = 5
  Average1 = 4
  Average2 = 5
  Low = 2
  NotLow = 4.5

  for i in range(len(data)):
    # for j in range(len(data[i])):
    x = float(data[i][2])
    #HIGH
    if (x > High):
      Hi = 1
    elif (x <= NotHigh):
      Hi = 0
    else:
      Hi = round((x-NotHigh)/(High-NotHigh),2)

    #LOW
    if (x <= Low):
      Lo = 1
    elif (x > NotLow):
      Lo = 0
    else:
      Lo = round((NotLow-x)/(NotLow-Low),2)

    #AVERAGE
    if ((x > Average1) & (x <= Average2)):
      Avg = 1
    elif ((x <= Low) or (x > High)):
      Avg = 0
    else:
      if (x > Average2):
        Avg = round((High-x)/(High-Average2),2)
      elif (x <= Average1):
        Avg = round((x-Low)/(Average1-Low),2)
    
    rat = []
    rate = []
    rat.append(Hi)   #HIGH     (indeks0)
    rat.append('high')
    rate.append(rat)
    rat = []
    rat.append(Avg)  #AVERAGE  (indeks1)
    rat.append('average')
    rate.append(rat)
    rat = []
    rat.append(Lo)   #LOW      (indeks2)
    rat.append('low')
    rate.append(rat)
    rat = []
    EngageRate.append(rate)
  
  return EngageRate

# **RULE INFERENSI**

In [0]:
def inferensi():
  # FOLLOWERS	| ENGAGEMENT RATE	|	SCORE
  # Upper		  | High			      | Chosen
  # Upper		  |  Average			  | Chosen
  # Upper		  |  Low			      | Considered
    
  # Middle		|  High			      | Chosen
  # Middle		|  Average			  | Considered
  # Middle		|  Low			      | Rejected

  # Bottom		|  High			      | Considered
  # Bottom		|  Average			  | Rejected
  # Bottom		|  Low			      | Rejected

  followers = fuzzificationFollowers()
  engageRate = fuzzificationEngageRate()
  concs = []

  for i in range(len(followers)):
    rules = [[followers[i][0],engageRate[i][0]],
             [followers[i][0],engageRate[i][1]],
             [followers[i][0],engageRate[i][2]],
             [followers[i][1],engageRate[i][0]],
             [followers[i][1],engageRate[i][1]],
             [followers[i][1],engageRate[i][2]],
             [followers[i][2],engageRate[i][0]],
             [followers[i][2],engageRate[i][1]],
             [followers[i][2],engageRate[i][2]]]
    
    Chosen = []
    Considered = []
    Rejected = []
    for z in range(len(rules)):
      if (rules[z][0][1] == 'upper'):
        if (rules[z][1][1] == 'high'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'average'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'low'):
          Considered.append(z)

      elif (rules[z][0][1] == 'middle'):
        if (rules[z][1][1] == 'high'):
          Chosen.append(z)
        elif (rules[z][1][1] == 'average'):
          Considered.append(z)
        elif (rules[z][1][1] == 'low'):
          Rejected.append(z)

      elif (rules[z][0][1] == 'bottom'):
        if (rules[z][1][1] == 'high'):
          Considered.append(z)
        elif (rules[z][1][1] == 'average'):
          Rejected.append(z)
        elif (rules[z][1][1] == 'low'):
          Rejected.append(z)
    
    for x in range(len(rules)):
      minn = min(rules[x])
      rules[x].append(minn)

    ch = max(rules[Chosen[0]][2][0],rules[Chosen[1]][2][0],rules[Chosen[2]][2][0])
    co = max(rules[Considered[0]][2][0],rules[Considered[1]][2][0],rules[Considered[2]][2][0])
    re = max(rules[Rejected[0]][2][0],rules[Rejected[1]][2][0],rules[Rejected[2]][2][0])
    conc = []
    conc.append(ch)
    conc.append(co)
    conc.append(re)
    concs.append(conc)
  # print("Ch",Chosen)
  # print("Co",Considered)
  # print("Re",Rejected)


    
  # print("HASIL Inferensi")
  # for k in range(len(concs)):
  #     print(concs[k])
  return concs


# **DEFUZZIFIKASI**

Using Takagi-Sugeno-style (constant defuzzification)

In [0]:
def defuzzification():
  inf = inferensi()
  hasilDef = []

  Chosen = 50
  Considered = 70
  Rejected = 100

  for i in range(len(inf)):
    Z = (100*inf[i][0])+(70*inf[i][1])+(50*inf[i][2])
    hasilDef.append(round(Z/sum(inf[i]),2))

  # for k in range(len(hasilDef)):
    # print(hasilDef[k])

  return hasilDef

# **SORTING**

Menentukan 20 Influencer Terbaik

In [0]:
def BestInfluencer():
  defuzzi = defuzzification()
  data = readFile()
  
  bestt = []

  ngurut = numpy.argsort(defuzzi)[::-1]
  for i in range(20):
    best = []
    best.append(data[ngurut[i]][0])
    bestt.append(best)
  for k in range(20):
    bestt[k].insert(0,k+1)
  for j in range(20):
    print(j+1,data[ngurut[j]])
  print(len(bestt))
  return bestt

# **WRITE TO CSV**

In [111]:
def writeCSV():
  best = BestInfluencer()
  f = open('chosen.csv', 'w')
  w = csv.writer(f)
  w.writerow(('No.','Nomor Record'))
  w.writerows(best)

writeCSV()

1 ['13', '90773', '6.7']
2 ['72', '61456', '4.6']
3 ['11', '95117', '6.6']
4 ['59', '58987', '6.7']
5 ['60', '58403', '5.2']
6 ['41', '81909', '3.6']
7 ['75', '94551', '3.0']
8 ['25', '77585', '2.7']
9 ['19', '55217', '4.9']
10 ['2', '38237', '5.8']
11 ['53', '56257', '3.2']
12 ['39', '9890', '9.4']
13 ['61', '53701', '4.5']
14 ['74', '22394', '6.3']
15 ['93', '27657', '5.5']
16 ['100', '15788', '6.8']
17 ['76', '39175', '3.3']
18 ['67', '31642', '3.4']
19 ['48', '19925', '6.5']
20 ['36', '30419', '3.3']
20
